In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import os
print(os.getcwd())
#os.chdir("/content/gdrive/Shared with me/offmychest/")
os.chdir("/content/gdrive/My Drive/offmychest/data_theo/")
print(os.getcwd())


/content/gdrive/My Drive/offmychest/data_theo/fastText-0.9.2
/content/gdrive/My Drive/offmychest/data_theo


In [ ]:
import re
import nltk

##Preprocessing the Data

Current state of the data: Author/Comment_Body/Label

Labels are based on ?

Assuming that they are based on whether or not the author is depressed, we need to:

* Remove the Author column (alt: replace each unique username with a number)
* Remove entries where Author is AutoModerator
* For every entry in Comment_Body - convert to all lowercase, get rid of special characters, break into 500-word chunks if required

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#load dictionary of english contractions
contractions = pd.read_json('/content/gdrive/My Drive/offmychest/data_theo/contractions.json', typ='series')
contractions = contractions.to_dict()

In [ ]:
contractions_regular_expression = re.compile('(%s)' % '|'.join(contractions.keys()))
symbols_regular_expression = re.compile('[^0-9a-z #+_]')
link_regular_expression = re.compile('https\S+|www\S+|http\S+')

In [ ]:
def expandContractions(text, c_re=contractions_regular_expression):
    def replace(match):
        return contractions[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def clean_posts(posts):
    cleaned_posts = []
    for post in posts:
        post = str(post)
        post = post.lower()
        post = symbols_regular_expression.sub(' ', post)
        post = link_regular_expression.sub(' ', post)
        post = expandContractions(post)
        post = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", post).split())
        stop_words = set(stopwords.words('english'))
        word_tokens = nltk.word_tokenize(post) 
        #lemmatizer
        # filtered_sentence = [stemmer.lemmatize(word) for word in word_tokens]
        # filtered_sentence = [stemmer2.stem(word) for word in filtered_sentence]
        # filtered_sentence = [w for w in filtered_sentence if not w in stop_words]
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        post = ' '.join(filtered_sentence)
        
        cleaned_posts.append(post)
        
    return cleaned_posts

In [ ]:
data = pd.read_csv ('dataset.csv',engine='python')
data.sample()

,Unnamed: 0,comment,label
130186,130185,“Miami won the 1st round”,1.0


In [ ]:
X, y = [],[]
for i in range(len(data)):
  if len(data['comment'][i])>200 and 'bing bong' not in data['comment'][i]:
    X.append(data['comment'][i])
    y.append(data['label'][i])

X = clean_posts([post for post in X])

In [ ]:
X[:10]

['think one biggest issues people driving aces away community especially many us love trans gender attracted buddies aces usually find identity lgbt events spaces etc even separate necessarily agree definitely need maybe merge sense least work together like said battling exclusion hurting sides',
 'actually yes everyone know lived grew mostly fond memories fact lot friends christian muslim people say loud pretty diverse',
 'kratom subtle gives maybe 20min motivation improved mood whole euphoric recreational buzz part away 1 2hrs max antidepressant effects like less negative thoughts phenibut contrary king doses 4 6 g make super social happy motivated creative sadly phenibut addiction forming sense harmful kratom',
 'would like see provision private women colleges hbcus want speak hbcu alum one know experiencing issues wcs worry public universities free remaining women colleges go already seeing institutions go co ed close high rates provide wonderful learning environment perhaps could 

In [ ]:
print(len(X))

31356


#Simple Model


In [ ]:
# Install Fasttext
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip

#Move to the fastText directory and build it:

%cd fastText-0.9.2
# for command line tool :
!make
# for python bindings :
!pip install .


--2021-05-20 18:52:20--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2021-05-20 18:52:21--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip.3’

v0.9.2.zip.3            [  <=>               ]   4.17M  11.4MB/s    in 0.4s    

2021-05-20 18:52:21 (11.4 MB/s) - ‘v0.9.2.zip.3’ saved [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
replace fastText-0.9.2/.circleci/cmake_test.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename:

In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 6.7MB/s 


Here we create the dataset as a text file for Fasttext.

In [ ]:
# Fasttext modeling
import fasttext
# import emoji




df_list = [(body,label) for body,label in zip(X,y)]

final_text = ''

for row in df_list:
  label = row[1]
  body = row[0]
  if type(body) != str:
    print('emoji here, install it please')
    #  body = emoji.demojize(body, delimiters=(' ', ' '))

  final_text += '__label__' + str(int(label)) + ' ' + str(body) + '\n'

data_text = open("data_text.txt", "w")
data_text.write(final_text)
data_text.close()




In [ ]:
# Opening a file
file = open("data_text.txt","r")
Counter = 0
  
# Reading from file
Content = file.read()
CoList = Content.split("\n")
  
for i in CoList:
    if i:
        Counter += 1
          
print("This is the number of lines in the file")
print(Counter)

This is the number of lines in the file
31356


In [ ]:
!head -n 21356 data_text.txt > data.train
!tail -n 10000 data_text.txt > data.valid

In [ ]:
fasttext_model = fasttext.train_supervised(input="data.train")
# fasttext_model = fasttext.train_supervised(input="data.train", wordNgrams=3, lr=0.5, epoch=10)

In [ ]:
fasttext_model.predict('I am so sad')
fasttext_model.predict('I am depressed')
# fasttext_model.predict('I am so happy')

(('__label__1',), array([1.00000203]))

In [ ]:
fasttext_model.test("data.valid")

(10000, 0.6275, 0.6275)

In [ ]:
fasttext_model_nowordgrams = fasttext.train_supervised(input="data.train", lr=0.5, epoch=15)

In [ ]:
fasttext_model_nowordgrams.test("data.valid")

(10000, 0.6222, 0.6222)

In [ ]:
fasttext_model_more_epochs = fasttext.train_supervised(input="data.train",epoch = 25)

In [ ]:
fasttext_model_more_epochs.test('data.valid')

(10000, 0.6235, 0.6235)

In [ ]:
fasttext_model_lr = fasttext.train_supervised(input="data.train", lr = 1.0)

In [ ]:
fasttext_model_lr.test('data.valid')

(10000, 0.6256, 0.6256)

In [ ]:
fasttext_model_more_epochs_lr = fasttext.train_supervised(input="data.train", epoch = 25, lr = 1.0)

In [ ]:
fasttext_model_more_epochs_lr.test('data.valid')

(10000, 0.6179, 0.6179)

In [ ]:
fasttext_model_wg3 = fasttext.train_supervised(input="data.train", lr = 0.1, wordNgrams = 2, epoch = 19)

In [ ]:
fasttext_model_wg3.test('data.valid')

(10000, 0.6414, 0.6414)

In [ ]:
fasttext_model_wg3.save_model("fasttext_model_good.bin")

In [ ]:
def feel(text):
  return 'I feel like ' + text
def sad(text):
  return text + ', which is sad'
def know(text):
  return 'i know '+text
def think(text):
  return 'i think ' + text
def really(text):
  _ = text.split()[:-1]
  _.append('really') 
  _.append(text.split()[-1])
  s=" "
  s = s.join(_)
  return s
def m_bias(text):
  return 'as a man, I think '+text
def f_bias(text):
  return 'as a woman, I think '+text
def life(text):
  return "all my life I've thought " + text 
def depress(text):
  return text + ', which is depressing'
def therapist(text):
  return "my therapist says that " + text

In [ ]:
methods = {'feel':feel,'sad':sad,'know':know,'think':think, 'really':really, 'm_bias': m_bias, 'f_bias':f_bias, 'life':life, 'depress':depress, 'therapist':therapist}

In [ ]:
texts = ['i have a bad feeling about this',"that dog is so ugly i'll throw it away","let's go away for the weekend",'may the force be with you','you are selfish','my sister is hungry','this is sparta','you are not ok','the world is mine','my friend died','i have explosive diarrhea','everyone is here','someone loves me', 'i can do that', 'you are a good person','i just talked to my mom', 'i miss my family', 'i love pasta', 'spider-man is my fav superhero','i love eating',"that's kinda cool",'this is gonna be a long day',"i hate going to the dentist","your music is so cool","i can't wait to go back home",'he cheated on me with my sister',"i just don't know what really happened",'i just got my period','i need to work on myself',"i'm just a big disappointment","he's pretty gay","it's not gay if you say no homo","chadwick boseman should've won the oscar","being alone is his biggest fear",'i can never look him in the eyes again','he is very handsome','you should just read a book',"he's a real gangster",'all i want is to netflix and chill with you','all i want for christmas is you',"my mom doesn't know i'm straight","i'm so jealous of hannah montana, she has everything",'my parents are getting a divorce',"i'm not having dinner with that son of a bitch",'my milkshake brings all the boys to the yard','babies are overrated',"i'm so wet right now","you shouldn't judge a book by its cover","she thinks she is not racist",'carrots are my thing']

In [ ]:
len(texts)

50

In [ ]:
def try_methods(_model):
  for method in methods:
    score = 0
    for text in texts:
      out = methods[method](text)
      if _model.predict(out)[0][0] == '__label__1':
        # print(method,1)
        score+=1
    print(method,': ',score/len(texts))

  

In [ ]:
try_methods(fasttext_model_wg3)

NameError: ignored

In [ ]:
# fasttext_model_wg3.predict('You are selfish')
# texts = ['you are selfish','my sister is hungry','this is sparta','you are not ok','the world is mine']
for text in texts:
  # test = [text, 'I feel like '+text,  text + ', which is sad', 'i know '+text, really(text), 'as a man, I think '+text,'as a woman, I think ' + text, 'all my life I have known ' + text ]
  #[text.split()[:-1]+['really']+text.split()[-1] ].join(' ')
  if fasttext_model_wg3.predict(text)[0][0]!='__label__0':
    print(text)



In [ ]:
test = [methods[method](text) for method in methods]
for method in test:
  _ = methods[method]('you are being selfish')
  print(_,fasttext_model_wg3.predict(_)[0][0])


NameError: ignored

In [ ]:
for text in texts:
  # test = [text, 'I feel like '+text,  text + ', which is sad', 'i know '+text, really(text), 'as a man, I think '+text,'as a woman, I think ' + text, 'all my life I have known ' + text ]
  #[text.split()[:-1]+['really']+text.split()[-1] ].join(' ')
  print('------>',text)
  test = [methods[method](text) for method in methods]
  for _ in test:
    if fasttext_model_wg3.predict(_)[0][0]=='__label__1':
      print(_)

------> i have a bad feeling about this
I feel like i have a bad feeling about this
all my life I've thought i have a bad feeling about this
i have a bad feeling about this, which is depressing
my therapist says that i have a bad feeling about this
------> that dog is so ugly i'll throw it away
that dog is so ugly i'll throw it away, which is depressing
my therapist says that that dog is so ugly i'll throw it away
------> let's go away for the weekend
I feel like let's go away for the weekend
let's go away for the weekend, which is sad
all my life I've thought let's go away for the weekend
let's go away for the weekend, which is depressing
my therapist says that let's go away for the weekend
------> may the force be with you
may the force be with you, which is depressing
my therapist says that may the force be with you
------> you are selfish
I feel like you are selfish
you are selfish, which is sad
i know you are selfish
you are really selfish
as a man, I think you are selfish
all my 

## Word Embeddings: FastText
Here, getting and saving word embeddings using FastText.

In [ ]:
# # Command line stuff - get the Wikipedia database & preprocess

# %mkdir data
# !wget -c http://mattmahoney.net/dc/enwik9.zip -P data
# !unzip data/enwik9.zip -d data

# !perl wikifil.pl data/enwik9 > data/fil9

# #We can check the file by running the following command:

# !head -c 80 data/fil9


In [ ]:
# # Python stuff - Train the representations

# import fasttext
# cbow_fasttext_vec_model = fasttext.train_unsupervised('data/fil9', "cbow")
# print("CBOW representations trained.")
# skipgram_fasttext_vec_model = fasttext.train_unsupervised('data/fil9', "skipgram")
# print("skipgram representations trained.")

# cbow_fasttext_vec_model.save_model("result/cbow_fil9.bin")
# skipgram_fasttext_vec_model.save_model("result/skipgram_fil9.bin")

In [ ]:
# # Visualize the representations - nearest neighbors, etc.

# model.get_nearest_neighbors('asparagus')

##Word Embeddings: BERT

##Training Experiments

In [ ]:
# import sklearn.linear_model

# def word_featurizer(xs):
#     # normalize
#     return xs / np.sqrt((xs ** 2).sum(axis=1, keepdims=True))

# def lsa_featurizer(xs):
#     # This function takes in a matrix in which each row contains the word counts
#     # for the given review. It should return a matrix in which each row contains
#     # the learned feature representation of each review (e.g. the sum of LSA 
#     # word representations).

#     td_matrix_tfidf = transform_tfidf(td_matrix)
#     reps_tfidf = learn_reps_lsa(td_matrix_tfidf, 500)

#     feats = np.matmul(xs, reps_tfidf)

#     # normalize
#     return feats / np.sqrt((feats ** 2).sum(axis=1, keepdims=True))

# # We've implemented the remainder of the training and evaluation pipeline,
# # so you likely won't need to modify the following four functions.
# def combo_featurizer(xs):
#     return np.concatenate((word_featurizer(xs), lsa_featurizer(xs)), axis=1)

# def train_model(featurizer, xs, ys):
#     xs_featurized = featurizer(xs)
#     model = sklearn.linear_model.LogisticRegression()
#     model.fit(xs_featurized, ys)
#     return model

# def eval_model(model, featurizer, xs, ys):
#     xs_featurized = featurizer(xs)
#     pred_ys = model.predict(xs_featurized)
#     return np.mean(pred_ys == ys)

# def training_experiment(name, featurizer, n_train):
#     print(f"{name} features, {n_train} examples")
#     train_xs = vectorizer.transform(train_reviews[:n_train])
#     train_ys = train_labels[:n_train]
#     test_xs = vectorizer.transform(test_reviews)
#     test_ys = test_labels
#     model = train_model(featurizer, train_xs, train_ys)
#     acc = eval_model(model, featurizer, test_xs, test_ys)
#     print(acc, '\n')
#     return acc

# # The following four lines will run a training experiment with all 3k examples
# # in training set for each feature type. `training_experiment` may be useful to
# # you when performing experiments to answer questions in Part 1 of the Homework
# # 1 handout.

# n_train = 3000
# training_experiment("word", word_featurizer, n_train)
# training_experiment("lsa", lsa_featurizer, n_train)
# training_experiment("combo", combo_featurizer, n_train)
# print()